In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("data_processed.csv")
df

,Giá,Năm sản xuất,Kiểu dáng,Xuất xứ,Số km đã đi,Hộp số,Nhiên liệu
0,1.119000e+09,2016,Sedan,Trongnước,51.499,Sốtựđộng,Xăng
1,1.789000e+09,2021,SUV,Nhậpkhẩu,8.000,Sốtựđộng,Xăng
2,5.200000e+08,2021,MPV,Nhậpkhẩu,10.000,Sốtay,Xăng
3,3.200000e+08,2009,Sedan,Trongnước,90.000,Sốtựđộng,Xăng
4,7.300000e+08,2016,SUV,Trongnước,68.000,Sốtựđộng,Xăng
...,...,...,...,...,...,...,...
2291,6.420000e+08,2018,Pick-upTruck,Nhậpkhẩu,15.000,Sốtựđộng,Diesel
2292,8.980000e+08,2018,Sedan,Trongnước,50.000,Sốtựđộng,Xăng
2293,3.800000e+08,2014,Hatchback,Nhậpkhẩu,92.000,Sốtựđộng,Xăng
2294,1.270000e+08,1993,Sedan,Nhậpkhẩu,68.000,Sốtựđộng,Xăng


In [6]:
df['Số km đã đi'] = df['Số km đã đi'].str.replace(' km', '').str.replace('.', '').astype(float)

In [7]:
print(df['Số km đã đi'])

0       51499.0
1        8000.0
2       10000.0
3       90000.0
4       68000.0
         ...   
2291    15000.0
2292    50000.0
2293    92000.0
2294    68000.0
2295     6000.0
Name: Số km đã đi, Length: 2296, dtype: float64


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2296 entries, 0 to 2295
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Giá           2296 non-null   float64
 1   Năm sản xuất  2296 non-null   int64  
 2   Kiểu dáng     2296 non-null   object 
 3   Xuất xứ       2296 non-null   object 
 4   Số km đã đi   2296 non-null   object 
 5   Hộp số        2296 non-null   object 
 6   Nhiên liệu    2296 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 125.7+ KB


In [6]:
print(df['Kiểu dáng'].unique())

['Sedan' 'SUV' 'MPV' 'Pick-upTruck' 'Hatchback' 'Convertible' 'CUV'
 'Coupe' 'Van/Minivan' 'SportCar' 'Truck']


In [7]:
print(df['Xuất xứ'].unique())

['Trongnước' 'Nhậpkhẩu']


In [8]:
print(df['Hộp số'].unique())

['Sốtựđộng' 'Sốtay' 'Sốhỗnhợp']


In [9]:
print(df['Nhiên liệu'].unique())

['Xăng' 'Diesel' 'Loạikhác']


Convert những giá trị chuỗi của cột  Kiểu dáng, Xuất xứ, Số km đã đi, Hộp số, Nhiên liệu sang giá trị số để train model

In [10]:
from sklearn.preprocessing import LabelEncoder

# Tạo thực thể LabelEncoder
le_kieudang = LabelEncoder()

# Danh sách các giá trị cần ánh xạ
categories_kieudang = ['Sedan', 'SUV', 'MPV', 'Pick-upTruck', 'Hatchback', 'Convertible', 'CUV',
                       'Coupe', 'Van/Minivan', 'SportCar', 'Truck']

# Huấn luyện LabelEncoder với các giá trị trên
le_kieudang.fit(categories_kieudang)

# Ánh xạ các giá trị này sang số nguyên bằng phương thức transform()
mapped_values = le_kieudang.transform(categories_kieudang)

# In ra ánh xạ giữa giá trị gốc và giá trị số nguyên tương ứng
for original, mapped in zip(categories_kieudang, mapped_values):
    print(f'{original}: {mapped}')

Sedan: 7
SUV: 6
MPV: 4
Pick-upTruck: 5
Hatchback: 3
Convertible: 1
CUV: 0
Coupe: 2
Van/Minivan: 10
SportCar: 8
Truck: 9


In [11]:
# Tạo thực thể LabelEncoder
le_nhapkhau = LabelEncoder()

# Danh sách các giá trị cần ánh xạ cho cột "Nhập khẩu"
categories_nhapkhau = ['Trongnước', 'Nhậpkhẩu']

# Huấn luyện LabelEncoder với các giá trị trên
le_nhapkhau.fit(categories_nhapkhau)

# Ánh xạ các giá trị này sang số nguyên bằng phương thức transform()
mapped_values_nhapkhau = le_nhapkhau.transform(categories_nhapkhau)

# In ra ánh xạ giữa giá trị gốc và giá trị số nguyên tương ứng
for original, mapped in zip(categories_nhapkhau, mapped_values_nhapkhau):
    print(f'{original}: {mapped}')

Trongnước: 1
Nhậpkhẩu: 0


In [12]:
# Tạo thực thể của LabelEncoder
le_hopso = LabelEncoder()

# Danh sách các giá trị cần ánh xạ cho cột "Hộp số"
categories_hopso = ['Sốtựđộng', 'Sốtay', 'Sốhỗnhợp']

# Huấn luyện LabelEncoder với các giá trị trên
le_hopso.fit(categories_hopso)
df['Hộp số'] = le_hopso.transform(df['Hộp số'])
# Ánh xạ các giá trị này sang số nguyên bằng phương thức transform()
mapped_values_hopso = le_hopso.transform(categories_hopso)

# In ra ánh xạ giữa giá trị gốc và giá trị số nguyên tương ứng
for original, mapped in zip(categories_hopso, mapped_values_hopso):
    print(f'{original}: {mapped}')

Sốtựđộng: 2
Sốtay: 1
Sốhỗnhợp: 0


In [13]:
# Tạo thực thể của LabelEncoder
le_nhienlieu = LabelEncoder()

# Danh sách các giá trị cần ánh xạ cho cột "Nhiên liệu"
categories_nhienlieu = ['Xăng', 'Diesel', 'Loạikhác']

# Huấn luyện LabelEncoder với các giá trị trên
le_nhienlieu.fit(categories_nhienlieu)

# Ánh xạ các giá trị này sang số nguyên bằng phương thức transform()
mapped_values_nhienlieu = le_nhienlieu.transform(categories_nhienlieu)

# In ra ánh xạ giữa giá trị gốc và giá trị số nguyên tương ứng
for original, mapped in zip(categories_nhienlieu, mapped_values_nhienlieu):
    print(f'{original}: {mapped}')

Xăng: 2
Diesel: 0
Loạikhác: 1


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2296 entries, 0 to 2295
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Giá           2296 non-null   float64
 1   Năm sản xuất  2296 non-null   int64  
 2   Kiểu dáng     2296 non-null   int32  
 3   Xuất xứ       2296 non-null   int32  
 4   Số km đã đi   2296 non-null   float64
 5   Hộp số        2296 non-null   int32  
 6   Nhiên liệu    2296 non-null   int32  
dtypes: float64(2), int32(4), int64(1)
memory usage: 89.8 KB


In [21]:
df['Kiểu dáng'] = df['Kiểu dáng'].replace({'Sedan': 1,'SUV':2,'MPV':3,'Pick-upTruck':4,'Hatchback':5,'Convertible':6,'CUV':7,'Coupe':8,'Van/Minivan':9,'SportCar':10,'Truck':11}).astype(int)

In [24]:
df['Xuất xứ'] = df['Xuất xứ'].replace({'Nhậpkhẩu': 1,'Trongnước':2}).astype(int)

In [26]:
df['Hộp số'] = df['Hộp số'].replace({'Sốhỗnhợp': 1,'Sốtay':2,'Sốtựđộng':3}).astype(int)

In [28]:
df['Nhiên liệu'] = df['Nhiên liệu'].replace({'Xăng': 1,'Diesel':2,'Loạikhác':3}).astype(int)

Đã xong chuẩn hóa dữ liệu

## Bắt đầu xây dựng mô hình

In [5]:

# # Đọc file data
df = pd.read_csv("data_processed.csv")

# Sửa lỗi chính tả
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "")

# Chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].astype(float) 

# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.2, random_state=42
)

# Huấn luyện mô hình
model = LinearRegression()
model.fit(X_train, y_train)

# Dự đoán giá xe với thông tin mới
new_car_data = {
    "Năm sản xuất": 2010,
    "Số km đã đi": '52900',
    "Kiểu dáng": "Sedan",
    "Xuất xứ": "Trong nước",
    "Hộp số": "Số tự động",
    "Nhiên liệu": "Xăng"
}

# Chuyển đổi dữ liệu mới sang dạng số
new_car_data_encoded = pd.DataFrame([new_car_data], columns=new_car_data.keys())
encoded_data = encoder.transform(new_car_data_encoded[categorical_cols]).toarray()  
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))  # Tạo DataFrame từ mảng NumPy


y_pred = model.predict(X_test)

# Tính các chỉ số đánh giá
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R² Score:", r2)

new_car_data_encoded = new_car_data_encoded.drop(categorical_cols, axis=1).join(encoded_df)  # Kết hợp DataFrame

# Dự đoán giá xe
predicted_price = model.predict(new_car_data_encoded)
formatted_price = format(predicted_price[0], ",.0f")  # Định dạng số với dấu "," phân cách hàng nghìn
print("Giá xe dự đoán:", formatted_price,"VND")

Giá xe dự đoán: 580,962,337 VND


Mean Absolute Error (MAE): 642071362.2042962
Mean Squared Error (MSE): 2.2463510889022792e+18
R² Score: 0.19535829193594811


In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Huấn luyện mô hình Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Đánh giá độ chính xác của mô hình
y_pred_rf = rf_model.predict(X_test_scaled)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("Random Forest - Mean Absolute Error (MAE):", mae_rf)
print("Random Forest - Mean Squared Error (MSE):", mse_rf)
print("Random Forest - R² Score:", r2_rf)



# K-Fold Cross-Validation
cv_scores = cross_val_score(rf_model, scaler.transform(df.drop("Giá", axis=1)), df["Giá"], cv=5, scoring='r2')
print("Cross-Validated R² Scores:", cv_scores)
print("Mean Cross-Validated R² Score:", cv_scores.mean())

# Dự đoán giá xe với thông tin mới
new_car_data = {
    "Năm sản xuất": 2010,
    "Số km đã đi": '52900',
    "Kiểu dáng": "Sedan",
    "Xuất xứ": "Trong nước",
    "Hộp số": "Số tự động",
    "Nhiên liệu": "Xăng"
}

# Chuyển đổi dữ liệu mới sang dạng số
new_car_data_encoded = pd.DataFrame([new_car_data], columns=new_car_data.keys())
encoded_data = encoder.transform(new_car_data_encoded[categorical_cols]).toarray()
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))
new_car_data_encoded = new_car_data_encoded.drop(categorical_cols, axis=1).join(encoded_df)

# Dự đoán giá xe
new_car_data_scaled = scaler.transform(new_car_data_encoded)
predicted_price_rf = rf_model.predict(new_car_data_scaled)
formatted_price_rf = format(predicted_price_rf[0], ",.0f")
accuracy_percentage = r2_rf * 100
print(f"Độ chính xác của mô hình Random Forest: {accuracy_percentage:.2f}%")
print("Giá xe dự đoán (Random Forest):", formatted_price_rf, "VND")


Random Forest - Mean Absolute Error (MAE): 392155010.5849902
Random Forest - Mean Squared Error (MSE): 1.7613706706869586e+18
Random Forest - R² Score: 0.36907800744181307
Cross-Validated R² Scores: [ 0.39279828  0.34084864 -0.40065675  0.99995315  0.99995644]
Mean Cross-Validated R² Score: 0.4665799523656496
Độ chính xác của mô hình Random Forest: 36.91%
Giá xe dự đoán (Random Forest): 1,876,510,000 VND


In [13]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
import pandas as pd

# Đọc file data
df = pd.read_csv("data_processed.csv")

# Sửa lỗi chính tả
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "")

# Chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].astype(float)

# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.2, random_state=42
)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Định nghĩa lưới các siêu tham số
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Khởi tạo mô hình Random Forest
rf = RandomForestRegressor(random_state=42)

# Định nghĩa k-fold cross-validation với số lượng folds là 10
cv = KFold(n_splits=10, shuffle=True, random_state=42)

# Grid search với cross-validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=cv, scoring='r2', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Kết quả tốt nhất từ grid search
best_rf = grid_search.best_estimator_
print("Best parameters found: ", grid_search.best_params_)

# Đánh giá mô hình với các siêu tham số tốt nhất
y_pred_best_rf = best_rf.predict(X_test_scaled)
mae_best_rf = mean_absolute_error(y_test, y_pred_best_rf)
mse_best_rf = mean_squared_error(y_test, y_pred_best_rf)
r2_best_rf = r2_score(y_test, y_pred_best_rf)

print("Tuned Random Forest - Mean Absolute Error (MAE):", mae_best_rf)
print("Tuned Random Forest - Mean Squared Error (MSE):", mse_best_rf)
print("Tuned Random Forest - R² Score:", r2_best_rf)

# Biểu diễn R² Score dưới dạng phần trăm
accuracy_percentage = r2_best_rf * 100
print("Độ chính xác của mô hình (R² Score): {:.2f}%".format(accuracy_percentage))

# K-Fold Cross-Validation với mô hình tốt nhất
cv_scores = cross_val_score(best_rf, scaler.transform(df.drop("Giá", axis=1)), df["Giá"], cv=cv, scoring='r2')
print("Cross-Validated R² Scores:", cv_scores)
print("Mean Cross-Validated R² Score:", cv_scores.mean())

# Dự đoán giá xe với mô hình tốt nhất
new_car_data = {
    "Năm sản xuất": 2010,
    "Số km đã đi": '52900',
    "Kiểu dáng": "Sedan",
    "Xuất xứ": "Trong nước",
    "Hộp số": "Số tự động",
    "Nhiên liệu": "Xăng"
}

# Chuyển đổi dữ liệu mới sang dạng số
new_car_data_encoded = pd.DataFrame([new_car_data], columns=new_car_data.keys())
encoded_data = encoder.transform(new_car_data_encoded[categorical_cols]).toarray()
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))
new_car_data_encoded = new_car_data_encoded.drop(categorical_cols, axis=1).join(encoded_df)

# Dự đoán giá xe
new_car_data_scaled = scaler.transform(new_car_data_encoded)
predicted_price_best_rf = best_rf.predict(new_car_data_scaled)
formatted_price_best_rf = format(predicted_price_best_rf[0], ",.0f")
print("Giá xe dự đoán (Tuned Random Forest):", formatted_price_best_rf, "VND")


KeyboardInterrupt: 